In [88]:
import illustris_python as il
import mistree as mist
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import astropy.units as u
import astropy.constants as c
import pandas as pd
import networkx as nx
import seaborn as sns
import scienceplots

from sklearn.neighbors import radius_neighbors_graph, KernelDensity
from sklearn.preprocessing import PowerTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from scipy.spatial import Delaunay
from scipy.spatial.distance import euclidean, minkowski

import Utilities as ut
import Network_stats as ns

plt.style.use(['science','no-latex'])


We can't use spearman's correlation coefficient to estimate the theoretical classification accuracy. That was wrong.

We can estimate it by calculating the Bayes error rate: E_Bayes

In [4]:
# Load data
test = ns.network(masscut=1e10)
G = test.subhalo_delauany_network(xyzplot=False)
test.cweb_classify(xyzplot=False)
test.network_stats_delaunay()

There are 97233 subhalos with stellar mass greater than 0.6774.
length before buffering:  97233
length after buffering:  97233


In [5]:
test.data

,Degree,Mean E.L.,Min E.L.,Max E.L.,Clustering,Density,Neigh Density,Target
Node ID,,,,,,,,
0,3.786575,0.252438,0.121074,0.610647,0.000323,185.880269,175.964597,3.0
1,3.439105,0.245650,0.158843,0.353270,0.000354,231.832891,282.636896,3.0
2,4.289986,0.428999,0.032403,0.740822,0.000867,34.527336,41.029794,3.0
3,5.261107,0.751587,0.044699,1.320098,0.001769,3.062777,1.367740,3.0
4,9.891149,0.760858,0.254148,1.141795,0.001110,4.098506,5.143066,3.0
...,...,...,...,...,...,...,...,...
97228,81.350501,3.697750,0.803879,11.299435,0.003283,0.003181,0.272363,3.0
97229,74.863507,4.990900,0.789464,18.656740,0.006422,0.000498,0.844022,2.0
97230,40.762152,2.397774,1.058297,3.451464,0.002639,0.033671,0.064352,2.0


In [8]:
features = test.data.iloc[:,:-1] # All columns except the last one
targets = test.data.iloc[:,-1] # The last column

# scaler = PowerTransformer()
scaler = PowerTransformer(method = 'box-cox')
features = pd.DataFrame(scaler.fit_transform(features), index=features.index, columns=features.columns)

In [26]:
targets

Node ID
0        3.0
1        3.0
2        3.0
3        3.0
4        3.0
        ... 
97228    3.0
97229    2.0
97230    2.0
97231    2.0
97232    2.0
Name: Target, Length: 97233, dtype: float64

In [66]:
# function to calculate class conditional probabilities P(X|Y=c) for each class
def class_conditional(X, bandwidth=0.1):
    '''
    Calculate the class conditional
    X: features
    bandwidth: bandwidth of the kernel density estimator
    '''
    kde = KernelDensity(kernel='gaussian', bandwidth=bandwidth)
    kde.fit(X)
    return kde

# function to calculate class priors P(Y=c)
def class_priors(Y):
    '''
    Calculate the class priors
    Y: targets
    '''
    priors = Y.value_counts(normalize=True)
    return np.array(priors)

# function to calculate the posterior probability P(Y=c|X)
def posterior(X, class_kdes, cp):
    '''
    Calculate the posterior probability
    X: features
    class_kdes: class conditional
    cp: class priors
    '''
    n_classes = len(class_kdes)
    n_samples = X.shape[0]

    densities = np.zeros((n_samples, n_classes)) # initialize array to hold densities

    # calculating P(X|Y=c) for each class
    for c in range(n_classes):
        densities[:, c] = np.exp(class_kdes[c].score_samples(X)) # log density to density

    # calculate the marginal
    evidence = np.dot(densities, cp)

    # calculate the posterior
    posteriors = densities * cp / evidence[:, None]
    return posteriors

# function for Bayes error rate
def bayes_error_rate(posteriors):
    '''
    Calculate the Bayes error rate
    '''
    max_posterior = np.max(posteriors, axis=1) # find the maximum posterior probability
    error_rate = 1 - np.mean(max_posterior) # calculate the error rate
    return error_rate


In [68]:
classes = np.unique(targets) # unique classes
cp = class_priors(targets) # P(Y) for each class
n_classes = len(classes) # number of classes

# calculate class conditional densities
class_kdes = []
for c in classes:
    X_c = features[targets == c] # features for class c
    kde_c = class_conditional(X_c, bandwidth=0.1) # class conditional for class c
    class_kdes.append(kde_c)

# calculate posterior probabilities
posteriors = posterior(features, class_kdes, cp)

In [74]:
# For the entire dataset

E_B = bayes_error_rate(posteriors)
print(f'Theoretical limit for accuracy from Bayes error rate: {100*(1 - E_B):.2f}%')

Theoretical limit for accuracy from Bayes error rate: 80.96%


In [76]:
train_x, test_x, train_y, test_y = train_test_split(features, targets, test_size=0.2, random_state=42, stratify=targets)
valid_x, test_x, valid_y, test_y = train_test_split(test_x, test_y, test_size=0.2, random_state=42, stratify=test_y)

In [77]:
train_x

,Degree,Mean E.L.,Min E.L.,Max E.L.,Clustering,Density,Neigh Density
Node ID,,,,,,,
36743,-1.496560,-1.330398,-1.885867,-1.529334,-0.971143,1.508512,0.521915
54267,0.507127,0.180703,0.846382,-0.094478,-0.389119,-0.141646,0.652976
73557,0.698338,0.788193,0.754615,0.519385,0.964531,-0.808316,-1.447810
27683,-0.527550,-0.478617,-1.761313,-0.223468,-0.359281,0.672899,0.646236
65946,0.055208,0.678203,1.079885,0.480397,2.133200,-0.466013,-1.764806
...,...,...,...,...,...,...,...
75702,-1.262349,-1.031227,-1.264117,-1.210059,-0.487036,1.051188,0.045092
72183,1.023838,1.189333,0.845927,1.455817,1.213390,-1.257102,-1.293649
4148,-0.707952,-1.107639,-0.100995,-0.869225,-1.389971,1.018890,1.314402


In [78]:
valid_x

,Degree,Mean E.L.,Min E.L.,Max E.L.,Clustering,Density,Neigh Density
Node ID,,,,,,,
92158,0.246056,0.678353,1.530833,0.236917,1.545516,-0.705360,-1.455619
62533,1.156414,1.017655,1.546115,1.213315,0.639718,-1.107307,-0.022620
11717,-1.868187,-1.882340,-1.020117,-2.084037,-1.771958,1.873933,1.500442
84027,0.262551,0.705341,0.080630,0.332149,1.265634,-0.573257,-1.424107
77138,-0.543527,-0.500258,0.210656,-0.750986,-0.172685,0.537998,0.561757
...,...,...,...,...,...,...,...
10578,-1.650260,-1.724506,-2.543429,-1.877239,-1.737896,1.638770,1.279734
59203,0.317714,0.532250,-0.225467,0.637620,0.937900,-0.693579,-0.567265
24020,-0.084240,-0.153668,-0.686981,0.101302,-0.070397,0.146414,0.400870


In [79]:
test_x

,Degree,Mean E.L.,Min E.L.,Max E.L.,Clustering,Density,Neigh Density
Node ID,,,,,,,
30772,-0.652316,-0.546489,-0.370915,-0.151680,-0.136707,0.469533,0.561374
51899,2.768069,3.148266,-1.899988,3.530328,2.203886,-1.509830,-1.531377
88873,0.374288,0.224418,0.447324,0.352660,-0.106462,-0.352463,0.497226
83798,0.326899,0.334096,0.917754,0.036266,0.201574,-0.228816,-0.648321
69420,1.547016,1.356470,0.983313,1.333799,0.769192,-1.613111,-0.567501
...,...,...,...,...,...,...,...
85605,1.856301,2.116774,0.884054,1.450208,1.664861,-2.383661,-1.948974
93566,0.744992,0.668362,1.468612,0.613715,0.568114,-0.783462,-0.315647
73967,1.593864,1.689421,1.533505,1.645803,1.235246,-2.018643,-1.501378


The bandwidth is a hyperparameter in KDE that controls the smooothness of the estimated density. A well-chosen bandwigth balances the trade-off between underfitting and overfitting:

- Small bandwidth: Creates highly detailed density estimate, but leads to overfitting (too spiky).

- Large bandwidth: Smooths out the density estimation, but it may lead to underfitting (too broad and inaccurate).

In [150]:
# now for a given test set

classes = np.unique(train_y) # unique classes
n_classes = len(classes) # number of classes
cp = class_priors(train_y) # P(Y) for each class

# calculate class conditional densities
class_kdes = []
for c in classes:
    X_c = train_x[train_y == c] # features for class c
    kde_c = class_conditional(X_c, bandwidth=0.10680004325145757) # class conditional for class c
    class_kdes.append(kde_c)

# calculate posterior probabilities
posteriors = posterior(test_x, class_kdes, cp) # test set

E_B = bayes_error_rate(posteriors)

print(f'Theoretical accuracy limit for test set from Bayes error rate: {100*(1 - E_B):.2f}%')

Theoretical accuracy limit for test set from Bayes error rate: 76.49%


In [146]:
# maximising log likelihood for optimal bandwidth
bandwidths = np.logspace(-1.0, -0.8, 15)

grid = GridSearchCV(KernelDensity(kernel='gaussian'), param_grid={'bandwidth': bandwidths}, cv=None, verbose=2)
grid.fit(train_x)

Fitting 5 folds for each of 15 candidates, totalling 75 fits
[CV] END ......................................bandwidth=0.1; total time=  22.3s
[CV] END ......................................bandwidth=0.1; total time=  22.0s
[CV] END ......................................bandwidth=0.1; total time=  21.9s
[CV] END ......................................bandwidth=0.1; total time=  22.2s
[CV] END ......................................bandwidth=0.1; total time=  22.4s
[CV] END .......................bandwidth=0.1033441063880556; total time=  22.4s
[CV] END .......................bandwidth=0.1033441063880556; total time=  22.1s
[CV] END .......................bandwidth=0.1033441063880556; total time=  22.3s
[CV] END .......................bandwidth=0.1033441063880556; total time=  22.4s
[CV] END .......................bandwidth=0.1033441063880556; total time=  22.0s
[CV] END ......................bandwidth=0.10680004325145757; total time=  22.2s
[CV] END ......................bandwidth=0.10680

GridSearchCV(estimator=KernelDensity(),
             param_grid={'bandwidth': array([0.1       , 0.10334411, 0.10680004, 0.11037155, 0.11406249,
       0.11787686, 0.12181879, 0.12589254, 0.13010252, 0.13445329,
       0.13894955, 0.14359617, 0.14839818, 0.15336077, 0.15848932])},
             verbose=2)

In [147]:
best_bandwidth = grid.best_params_['bandwidth']

In [148]:
best_bandwidth

0.10680004325145757

In [149]:
np.logspace(-1.0, -0.8, 15)

array([0.1       , 0.10334411, 0.10680004, 0.11037155, 0.11406249,
       0.11787686, 0.12181879, 0.12589254, 0.13010252, 0.13445329,
       0.13894955, 0.14359617, 0.14839818, 0.15336077, 0.15848932])

In [133]:
10**np.linspace(-1.0, -0.5, 5)

array([0.1       , 0.13335214, 0.17782794, 0.23713737, 0.31622777])